In [2]:
import re

def extract_variables_section(text):
    
    variables_section_pattern = r"\*\*\* Variables \*\*\*(.*?)(?=\n\*\*\*|\Z)"
    match = re.search(variables_section_pattern, text, re.DOTALL)
    if match:
        return match.group(1).strip()
    return ""


def extract_resources(text) :
    content = rfc

    settings_section_pattern = r"\*\*\* Settings \*\*\*(.*?)(?=\n\*\*\*|\Z)"
    library_pattern = r"Library\s+(.+)"
    resource_pattern = r"Resource\s+(.+)"
    variables_pattern = r"Variables\s+(.+)"
    settings_section_match = re.search(settings_section_pattern, content, re.DOTALL)
    if settings_section_match:
        settings_section = settings_section_match.group(1)
        libraries = re.findall(library_pattern, settings_section)
        resources = re.findall(resource_pattern, settings_section)
        variables = re.findall(variables_pattern, settings_section)
        return (libraries , resources , variables)
    else:
        print("No *** Settings *** section found.")
        return ([],[],[])
    
def extract_keywords_section(text):
    keywords_section_pattern = r"\*\*\* Keywords \*\*\*(.*?)(?=\n\*\*\*|\Z)"
    match = re.search(keywords_section_pattern, text, re.DOTALL)
    if match:
        return match.group(1).strip()
    return ""

def extract_sttl_content_and_suite_sections(text, sttl_ids_to_filter):
    sttl_pattern = r"(STTL-\d+)(.*?)(?=\nSTTL-\d+|\n\*|\Z)"
    matches = re.findall(sttl_pattern, text, re.DOTALL)
    test_cases = {}
    for match in matches:
        sttl_id, content = match
        if sttl_id in sttl_ids_to_filter:
            test_cases[sttl_id] = f"{sttl_id}{content}".strip()
    

    suite_setup_pattern = r"\*\*\* Settings \*\*\*.*?Suite Setup\s+(.*?)(?=\n\w|\Z)"
    suite_teardown_pattern = r"\*\*\* Settings \*\*\*.*?Suite Teardown\s+(.*?)(?=\n\w|\Z)"
    test_setup_pattern = r"\*\*\* Settings \*\*\*.*?Test Setup\s+(.*?)(?=\n\w|\Z)"
    test_teardown_pattern = r"\*\*\* Settings \*\*\*.*?Test Teardown\s+(.*?)(?=\n\w|\Z)"
    test_timeout_pattern = r"\*\*\* Settings \*\*\*.*?Test Timeout\s+(.*?)(?=\r?\n|$)" 
    
    suite_setup_match = re.search(suite_setup_pattern, text, re.DOTALL)
    suite_teardown_match = re.search(suite_teardown_pattern, text, re.DOTALL)
    test_setup_match = re.search(test_setup_pattern, text, re.DOTALL)
    test_teardown_match = re.search(test_teardown_pattern, text, re.DOTALL)
    test_timeout_match = re.search(test_timeout_pattern, text, re.DOTALL)


    suite_setup = suite_setup_match.group(1).strip() if suite_setup_match else "Not found"
    suite_teardown = suite_teardown_match.group(1).strip() if suite_teardown_match else "Not found"
    test_setup = test_setup_match.group(1).strip() if test_setup_match else "Not found"
    test_teardown = test_teardown_match.group(1).strip() if test_teardown_match else "Not found"
    test_timeout = test_timeout_match.group(1).strip() if test_timeout_match else "Not found"


    result = {
        'Suite_Setup': suite_setup,
        'Suite_Teardown': suite_teardown,
        'Test_Setup': test_setup,
        'Test_Teardown': test_teardown,
        'Test_Timeout': test_timeout,
        'Test_Cases': test_cases 
    }
    return result

file = f"C:/RFS_CI/GIT_REPO/ST_Master/mcd_validation_RFS/RFS/TS/ANDROID/Android_OSUpdate/OS_Update3.0_Test.robot"

with open(file, "r") as rf:
    rfc = rf.read()

sttl_ids_to_filter = ["STTL-232540", "STTL-230890", "STTL-230895", "STTL-230899", "STTL-230907","STTL-244060","STTL-230904","STTL-231747"] 
extracted_content = extract_sttl_content_and_suite_sections(rfc, sttl_ids_to_filter)
extracted_content['Libraries'] = extract_resources(rfc)[0]
extracted_content['Resources'] = extract_resources(rfc)[1]
extracted_content['Variables'] = extract_resources(rfc)[2]
extracted_content['Keywords'] = extract_keywords_section(rfc)
extracted_content['Variables'] = extract_variables_section(rfc)

In [3]:
for key,value in extracted_content.items():
    print("-------------------------" , key , "-------------------------")
    print(value)
    

------------------------- Suite_Setup -------------------------
Suite_Setup_OSUpdate
------------------------- Suite_Teardown -------------------------
run keywords
...  Perform Factory Reset
...  AND  Enable Setup for Automation  ${DUT1}
------------------------- Test_Setup -------------------------
run keywords
...  Battery Setup For Staging
...  AND  Install  ${EXTERNALFILES_DIRECTORY}\\Android\\APK\\StageNow\\Generic\\BarcodeStaging.apk
...  AND  BuiltIn.Sleep  10
------------------------- Test_Teardown -------------------------
run keywords
...  Set Serial Number Of Device For All Libraries  ${DUT1}
...  AND  run keyword and ignore error  Unlock The Device Screen For All Security Locks
...  AND  run keyword and ignore error  Remove The Screen Lock Through Intent
------------------------- Test_Timeout -------------------------
400 minutes
------------------------- Test_Cases -------------------------
{'STTL-232540': "STTL-232540 - Pre Build Package --UPL- StageNow- Special packages

In [4]:
len(extracted_content['Test_Cases'])

8

In [5]:
import os 
from dotenv import load_dotenv
from langchain_openai import AzureChatOpenAI

def get_llm():
    load_dotenv(override=True)
    azure_deployment = os.environ.get('GPT4O_DEPLOYMENT_NAME')
    api_version = os.environ.get('GPT4O_API_VERSION')
    api_key = os.environ.get('GPT4O_API_KEY')
    azure_endpoint = os.environ.get('GPT4O_ENDPOINT')

    return AzureChatOpenAI(azure_deployment=azure_deployment,
openai_api_version=api_version,
openai_api_key=api_key,
azure_endpoint=azure_endpoint,
temperature=0)

def get_llm_response(prompt, llm=get_llm()):
    message = [
        {
            "role": "system",
            "content": "You are an expert at robot framework "
        },
        {
            "role": "user",
            "content": prompt
        }
    ]
    
    response = llm(messages=message)
    
    if hasattr(response, "content"):
        return response.content.strip()
    
    if isinstance(response, dict) and "choices" in response:
        if len(response["choices"]) > 0 and "message" in response["choices"][0]:
            return response["choices"][0]["message"]["content"].strip()
    
    return "No response content available."

In [12]:
prompt = ''' 
You are an expert in Robot Framework. I have the following components of a Robot Framework file stored in a dictionary:

For any reference about the indentation and the spacing you can refer this example robot file attached : {} . 


1. **Suite Setup**: The steps for the suite setup.
2. **Suite Teardown**: The steps for the suite teardown.
3. **Test Setup**: The steps for the test setup.
4. **Test Teardown**: The steps for the test teardown.
5. **Test Timeout**: The timeout value for each test case.
6. **Test Cases**: A dictionary where each key is a test case ID, and the value is the content of the test case.
7. **Libraries**: A list of libraries to import.
8. **Resources**: A list of resources to import.
9. **Variables**: The variables section of the file.
10. **Keywords**: The keywords section of the file.

The above mentioned components are attached here : {} .  Dont omit any test case , make sure all of them are present in the final robot file . 

Your task is to generate a complete Robot Framework file using these components alone , dont add your own stuff to any component. Follow these rules:

1. **Test Setup and Teardown**:
   - If a test case already has a `[Setup]` field, leave it unchanged. Otherwise create it and add the Test Setup steps to the `[Setup]` field.
   - If a test case already has a `[Teardown]` field, leave it unchanged. Otherwise create it and add the Test Teardown steps to the `[Teardown]` field. 

2. **Suite Setup and Teardown**:
   - Add the Suite Setup steps to the `[Setup]` field of the first test case, ensuring it appears before any existing setup.
   - Add the Suite Teardown steps to the `[Teardown]` field of the last test case, ensuring it appears after any existing teardown.
   - Make sure these are not added globally . 
   - These should be added only once each . 

3. **Test Timeout**:
   - Append the Test Timeout value to the end of every test case.
   - Dont add this globally . 
   - Dont add this to the last test case . 

4. **Formatting**:
   - Maintain proper indentation and formatting for Robot Framework files , use the example robot file for the reference of the same . 
   - USe AND to join any Teardowns or Setups . 

5. **Keywords**:
   - Add the keywords section at the end of the file . 

Output only the complete Robot Framework file as a single code block and nothing else , ensuring it adheres to the above rules and is ready to use. 
'''

In [13]:
with open("OS_Update3.0_Test.robot","r") as file : 
    example_robot = file.read()

response = get_llm_response(prompt.format(example_robot , extracted_content))

In [14]:
print(response)

```robot
*** Settings ***
Library  Collections
Library  ImageRecognitionLibrary

Resource  ${EXECDIR}/Res/Android/AndroidGMS/AndroidGMS_Res.robot
Resource  ${EXECDIR}/Res/Android/System/System_UIAutomator_Res.robot
Resource  ${EXECDIR}/Res/Android/EMMTK/EMMTK_Keywords.robot
Resource  ${EXECDIR}/Res/Android/DeviceImaging/DeviceImaging_UIAutomator_Res.robot
Resource  ${EXECDIR}/Res/Common/NetworkTraffic/Ping_Res.robot
Resource  ${EXECDIR}/Res/Android/Memory/AndroidMemory_UIAutomator_Res.robot
Resource  ${EXECDIR}/Res/Android/RCR_Form/Parse_RCR_Form_Res.robot
Resource  ${EXECDIR}/Res/Android/OOB/OOB_Settings_Res.robot
Resource  ${EXECDIR}/Res/Android/Browser/Browser_UIAutomator_Res.robot
Resource  ${EXECDIR}/Res/Android/EmdkScanner/Scanner_Injection_Common_Res.robot
Resource  ${EXECDIR}/Res/Android/AndroidGMSLite/AndroidGMSLite_UIAutomator_Res.robot
Resource  ${EXECDIR}/Res/Android/Android_Scan_Stress/Android_Scan_Stress_Res.robot
Resource  ${EXECDIR}/Res/Android/Android/AEF/AEF_UIAutomat

In [15]:
def remove_first_and_last_line(text):
    lines = text.strip().split("\n")
    if len(lines) > 2:
        return "\n".join(lines[1:-1])
    return "" 
response_filtered = remove_first_and_last_line(response)

In [16]:
with open("response.robot","w") as file : 
    file.write(response_filtered)